In [1]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine import create_engine

# Config
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
class DataBaseSession:
    def __init__(self, db_name: str) -> None:
        self.db_user = 'root'
        self.db_password = 'rushi12345'
        self.db_host = '192.168.29.7'
        self.db_name = db_name
    
    def LocalSession(self) -> sessionmaker:
        engine = create_engine(f"mysql+pymysql://{self.db_user}:{self.db_password}@{self.db_host}/{self.db_name}")
        SessionLocal = sessionmaker(autoflush=True, bind=engine)
        return SessionLocal()

In [3]:
session = DataBaseSession('test_partition')
db = session.LocalSession()

In [4]:
class SQL8:
    def __init__(self) -> None:
        pass
    
    def query(self, query: str) -> pd.DataFrame:
        return pd.DataFrame(db.execute(text(query)).fetchall())

In [5]:
data = SQL8()

In [6]:
data.query("""
show tables;
""")

,Tables_in_test_partition
0,course
1,course1
2,course2


In [7]:
db.execute(text("""
create table course(
course_id int NOT NULL,
course_name varchar(255),
course_title varchar(255),
course_desc varchar(255),
launch_date date,
course_fee decimal(20, 6),
course_mentor varchar(255),
course_launch_year int);
"""))

In [8]:
db.execute(text("""
commit;
"""))

In [10]:
data.query("""select * from course""")

""


1) What is partition in SQL?
   
   In SQL, a partition is a division of a large dataset into smaller, more manageable parts. This division is based on a      specific column or expression, and each partition can be processed independently.
   
2) Why do we use partition in SQL?
   
   Partitions are used for various purposes, including:

   * Performance optimization: By partitioning a large table, you can improve query performance, as queries only need to   process a subset of the data rather than the entire table.
   * Data management: Partitions can help manage data more efficiently, especially in scenarios where data needs to be   archived or purged regularly.
   * Maintenance operations: Partitions can make certain maintenance operations, such as index rebuilds or backup operations,    more efficient by operating on smaller, manageable partitions instead of the entire dataset.

1) What is partition in SQL?
A partition in SQL is a way to divide a large table into smaller, more manageable parts based on a specific column or expression. Each part is called a partition and behaves like a separate table, allowing for easier data management and improved query performance.

2) Why do we use partition in SQL?
Partitions are used in SQL for several reasons:

Improved Performance: By dividing data into smaller partitions, SQL can perform operations such as searches, updates, and deletions more efficiently.
Easier Data Management: Partitions allow for easier management of large datasets, such as archiving old data or moving partitions to different storage devices.
Query Optimization: Queries that involve partitioned tables can be optimized to only search relevant partitions, improving overall query performance.
3) Example of code in SQL and show tables to understand the concept instead of writing create table statement
   
   Consider a table sales_data with the following columns: date, product_id, sales_amount.

   Let's partition this table by date into monthly partitions. Each partition will contain data for a specific month.

   Before Partitioning:
   ```sql
   SELECT * FROM sales_data;
   ```
   
   ```table
   date       | product_id | sales_amount
   -----------|------------|-------------
   2024-01-01 | 1          | 100
   2024-01-05 | 2          | 150
   2024-02-10 | 1          | 200
   2024-02-15 | 3          | 120
   ```
   
   After Partitioning:
  
  ```sql
  SELECT * FROM sales_data PARTITION BY RANGE (TO_DAYS(date)) (
     PARTITION p0 VALUES LESS THAN (TO_DAYS('2024-02-01')),
     PARTITION p1 VALUES LESS THAN (TO_DAYS('2024-03-01')),
     PARTITION p2 VALUES LESS THAN (TO_DAYS('2024-04-01'))
  );
  ```

4) What are types of partitions?
   There are several types of partitions in SQL, including:

   * Range Partitioning: Divides data based on a specified range of values.
   * List Partitioning: Divides data based on a specified list of values.
   * Hash Partitioning: Divides data based on a hash function applied to a column's value.
   * Composite Partitioning: Combines two or more partitioning methods.
   
   Example of Each Partition:

   * Range Partitioning Example:
     Partition sales_data by date into quarterly partitions:

   ```sql
   PARTITION BY RANGE (TO_DAYS(date)) (
     PARTITION q1 VALUES LESS THAN (TO_DAYS('2024-04-01')),
     PARTITION q2 VALUES LESS THAN (TO_DAYS('2024-07-01')),
     PARTITION q3 VALUES LESS THAN (TO_DAYS('2024-10-01')),
     PARTITION q4 VALUES LESS THAN (TO_DAYS('2025-01-01'))
   )
   ```
   * List Partitioning Example:
     Partition sales_data by product_id into specific product partitions:

   ```sql
   PARTITION BY LIST (product_id) (
     PARTITION p1 VALUES IN (1, 2),
     PARTITION p2 VALUES IN (3, 4),
     PARTITION p3 VALUES IN (5, 6)
   )
   ```

   * Hash Partitioning Example:
     Partition sales_data by a hash of product_id into 4 partitions:

   ```sql
   PARTITION BY HASH(product_id) PARTITIONS 4
   Composite Partitioning Example:
   Combine range and hash partitioning for sales_data:
   ```
   
   ```sql
   PARTITION BY RANGE (TO_DAYS(date)) (
     PARTITION p202401 VALUES LESS THAN (TO_DAYS('2024-02-01')) PARTITION BY HASH(product_id) PARTITIONS 4,
     PARTITION p202402 VALUES LESS THAN (TO_DAYS('2024-03-01')) PARTITION BY HASH(product_id) PARTITIONS 4)
   ```

In [13]:
db.execute(text("""
insert into course values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [7]:
data.query("""
select * from course;
""")

,course_id,course_name,course_title,course_desc,launch_date,course_fee,course_mentor,course_launch_year
0,100,ML,Machine Learning,End to End Machine Learning course with captso...,2019-06-07,3999.990000,person1,2019
1,101,ML1.0,Machine Learning,End to End Machine Learning course with captso...,2019-07-07,3999.990000,person2,2019
2,102,ML2.0,Machine Learning,End to End Machine Learning course with captso...,2019-08-07,3999.990000,person3,2019
3,103,Ml3.0,Machine Learning,End to End Machine Learning course with captso...,2019-09-07,3999.990000,person1,2019
4,104,ML4.0,Machine Learning,End to End Machine Learning course with captso...,2020-07-07,3999.990000,person2,2020
5,105,ML5.0,Machine Learning,End to End Machine Learning course with captso...,2020-08-07,3999.990000,person3,2020
6,106,ML6.0,Machine Learning,End to End Machine Learning course with captso...,2020-09-07,3999.990000,person1,2020
7,107,ML7.0,Machine Learning,End to End Machine Learning course with captso...,2021-07-07,3999.990000,person2,2021
8,108,ML8.0,Machine Learning,End to End Machine Learning course with captso...,2021-08-07,3999.990000,person3,2021
9,109,ML9.0,Machine Learning,End to End Machine Learning course with captso...,2021-09-07,3999.990000,person1,2021


In [8]:
data.query("""
select * from course where course_launch_year = 2020;
""")

,course_id,course_name,course_title,course_desc,launch_date,course_fee,course_mentor,course_launch_year
0,104,ML4.0,Machine Learning,End to End Machine Learning course with captso...,2020-07-07,3999.990000,person2,2020
1,105,ML5.0,Machine Learning,End to End Machine Learning course with captso...,2020-08-07,3999.990000,person3,2020
2,106,ML6.0,Machine Learning,End to End Machine Learning course with captso...,2020-09-07,3999.990000,person1,2020


In [27]:
db.execute(text("""
CREATE TABLE course1 (
    course_id INT NOT NULL,
    course_name VARCHAR(255),
    course_title VARCHAR(255),
    course_desc VARCHAR(255),
    launch_date DATE,
    course_fee DECIMAL(20, 6),
    course_mentor VARCHAR(255),
    course_launch_year INT);
"""))

** Creating Partition in tables

In [29]:
db.execute(text("""
ALTER TABLE course1
PARTITION BY RANGE (course_launch_year) (
    PARTITION p0 VALUES LESS THAN (2020),
    PARTITION p1 VALUES LESS THAN (2021),
    PARTITION p2 VALUES LESS THAN (2022),
    PARTITION p3 VALUES LESS THAN (2023)
);
"""))

In [30]:
db.execute(text("""
commit;
"""))

In [31]:
db.execute(text("""
insert into course1 values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [43]:
db.execute(text("""
commit;
"""))

In [15]:
%%time
data.query("""
select * from course where course_launch_year = 2021;
""")

CPU times: total: 0 ns
Wall time: 3.86 ms


,course_id,course_name,course_title,course_desc,launch_date,course_fee,course_mentor,course_launch_year
0,107,ML7.0,Machine Learning,End to End Machine Learning course with captso...,2021-07-07,3999.990000,person2,2021
1,108,ML8.0,Machine Learning,End to End Machine Learning course with captso...,2021-08-07,3999.990000,person3,2021
2,109,ML9.0,Machine Learning,End to End Machine Learning course with captso...,2021-09-07,3999.990000,person1,2021


In [16]:
%%time
data.query("""
select * from course1 where course_launch_year = 2021;
""")

CPU times: total: 0 ns
Wall time: 2.55 ms


,course_id,course_name,course_title,course_desc,launch_date,course_fee,course_mentor,course_launch_year
0,107,ML7.0,Machine Learning,End to End Machine Learning course with captso...,2021-07-07,3999.990000,person2,2021
1,108,ML8.0,Machine Learning,End to End Machine Learning course with captso...,2021-08-07,3999.990000,person3,2021
2,109,ML9.0,Machine Learning,End to End Machine Learning course with captso...,2021-09-07,3999.990000,person1,2021


In [20]:
data.query("""
select partition_name , table_name , table_rows from information_schema.partitions where table_name = 'course1';
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course1,4
1,p1,course1,3
2,p2,course1,3
3,p3,course1,3


### Hashing Partition

In [38]:
db.execute(text("""
CREATE TABLE course2 (
    course_id INT NOT NULL,
    course_name VARCHAR(255),
    course_title VARCHAR(255),
    course_desc VARCHAR(255),
    launch_date DATE,
    course_fee DECIMAL(20, 6),
    course_mentor VARCHAR(255),
    course_launch_year INT);
"""))

In [39]:
db.execute(text("""
ALTER TABLE course2
PARTITION BY HASH(course_launch_year)
PARTITIONS 5;
"""))

In [40]:
db.execute(text("""
commit;
"""))

In [41]:
data.query("""
select partition_name, table_name, table_rows from information_schema.partitions where table_name = 'course2';
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course2,0
1,p1,course2,0
2,p2,course2,0
3,p3,course2,0
4,p4,course2,0


In [42]:
db.execute(text("""
insert into course2 values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [43]:
db.execute(text("""
commit;
"""))

In [44]:
data.query("""
select partition_name, table_name, table_rows from information_schema.partitions where table_name = 'course2';
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course2,3
1,p1,course2,3
2,p2,course2,3
3,p3,course2,0
4,p4,course2,4


* Create partation by hash by taking primary key column as input

```sql
ALTER TABLE course2
PARTITION BY KEY()
PARTITIONS 5;
```

In [47]:
data.query("""
select MD5("Hello World");
""")

,"MD5(""Hello World"")"
0,b10a8db164e0754105b7a99be72e3fe5


### List Partition

In [13]:
db.execute(text("""
CREATE TABLE course3 (
    course_id INT NOT NULL,
    course_name VARCHAR(255),
    course_title VARCHAR(255),
    course_desc VARCHAR(255),
    launch_date DATE,
    course_fee DECIMAL(20, 6),
    course_mentor VARCHAR(255),
    course_launch_year INT);
"""))

In [14]:
db.execute(text("""
ALTER TABLE course3
PARTITION BY list(course_launch_year)(
    PARTITION p0 VALUES IN (2019, 2020),
    PARTITION p1 VALUES IN (2021, 2022)
)
"""))

In [15]:
db.execute(text("""
commit;
"""))

In [16]:
db.execute(text("""
insert into course3 values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [17]:
db.execute(text("""
commit;
"""))

In [19]:
data.query("""
select partition_name, table_name, table_rows from information_schema.partitions where table_name = 'course3';
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course3,7
1,p1,course3,6


### Range Column Partation

* Multiple Columns as input to create partition

In [20]:
db.execute(text("""
CREATE TABLE course4 (
    course_id INT NOT NULL,
    course_name VARCHAR(255),
    course_title VARCHAR(255),
    course_desc VARCHAR(255),
    launch_date DATE,
    course_fee DECIMAL(20, 6),
    course_mentor VARCHAR(255),
    course_launch_year INT);
"""))

In [23]:
db.execute(text("""
ALTER TABLE course4
PARTITION BY RANGE COLUMNS(course_mentor, course_launch_year)(
    PARTITION p0 VALUES LESS THAN ("person1", 2020),
    PARTITION p1 VALUES LESS THAN ("person2", 2021),
    PARTITION p2 VALUES LESS THAN ("person3", 2023));
"""))

In [26]:
db.execute(text("""
insert into course4 values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [27]:
data.query("""
select partition_name, table_name, table_rows from information_schema.partitions where table_name = "course4";
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course4,2
1,p1,course4,5
2,p2,course4,6


In [36]:
data.query("""
SELECT * FROM course4 PARTITION (p2);
""")

,course_id,course_name,course_title,course_desc,launch_date,course_fee,course_mentor,course_launch_year
0,102,ML2.0,Machine Learning,End to End Machine Learning course with captso...,2019-08-07,3999.990000,person3,2019
1,105,ML5.0,Machine Learning,End to End Machine Learning course with captso...,2020-08-07,3999.990000,person3,2020
2,107,ML7.0,Machine Learning,End to End Machine Learning course with captso...,2021-07-07,3999.990000,person2,2021
3,108,ML8.0,Machine Learning,End to End Machine Learning course with captso...,2021-08-07,3999.990000,person3,2021
4,110,ML10.0,Machine Learning,End to End Machine Learning course with captso...,2022-07-07,3999.990000,person2,2022
5,111,ML11.0,Machine Learning,End to End Machine Learning course with captso...,2022-08-07,3999.990000,person3,2022


### List of columns

* Multiple column values

* **List** accept only int values but **List Columns** accept varity range of values 

In [37]:
db.execute(text("""
CREATE TABLE course5 (
    course_id INT NOT NULL,
    course_name VARCHAR(255),
    course_title VARCHAR(255),
    course_desc VARCHAR(255),
    launch_date DATE,
    course_fee DECIMAL(20, 6),
    course_mentor VARCHAR(255),
    course_launch_year INT);
"""))

In [38]:
db.execute(text("""
ALTER TABLE course5
partition by list columns(course_mentor) (
    partition p0 values in("person1"),
    partition p1 values in("person2"),
    partition p2 values in("person3")
)
"""))

In [39]:
db.execute(text("""
insert into course5 values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [40]:
data.query("""
select partition_name, table_name, table_rows from information_schema.partitions where table_name = "course5";
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course5,5
1,p1,course5,4
2,p2,course5,4


In [41]:
db.execute(text("""
commit;
"""))

### Partitions inside of partition

In [55]:
db.execute(text("""
CREATE TABLE course6 (
    course_id INT NOT NULL,
    course_name VARCHAR(255),
    course_title VARCHAR(255),
    course_desc VARCHAR(255),
    launch_date DATE,
    course_fee DECIMAL(20, 6),
    course_mentor VARCHAR(255),
    course_launch_year INT);
"""))

In [56]:
db.execute(text("""
ALTER TABLE course6
PARTITION BY RANGE (course_launch_year)
SUBPARTITION BY HASH(course_launch_year)
SUBPARTITIONS 4 (
    PARTITION p0 VALUES LESS THAN (2020),
    PARTITION p1 VALUES LESS THAN (2021),
    PARTITION p2 VALUES LESS THAN (2022),
    PARTITION p3 VALUES LESS THAN (2023)
);
"""))

In [57]:
db.execute(text("""
insert into course6 values (100, "ML", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-06-07', 3999.99, "person1", 2019),
(101, "ML1.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-07-07', 3999.99, "person2", 2019),
(102, "ML2.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-08-07', 3999.99, "person3", 2019),
(103, "Ml3.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2019-09-07', 3999.99, "person1", 2019),
(104, "ML4.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-07-07', 3999.99, "person2", 2020),
(105, "ML5.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-08-07', 3999.99, "person3", 2020),
(106, "ML6.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2020-09-07', 3999.99, "person1", 2020),
(107, "ML7.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-07-07', 3999.99, "person2", 2021),
(108, "ML8.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-08-07', 3999.99, "person3", 2021),
(109, "ML9.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2021-09-07', 3999.99, "person1", 2021),
(110, "ML10.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-07-07', 3999.99, "person2", 2022),
(111, "ML11.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-08-07', 3999.99, "person3", 2022),
(112, "ML12.0", "Machine Learning", "End to End Machine Learning course with captsone projects", '2022-09-07', 3999.99, "person1", 2022)
"""))

In [58]:
data.query("""
select partition_name, table_name, table_rows from information_schema.partitions where table_name = "course6";
""")

,PARTITION_NAME,TABLE_NAME,TABLE_ROWS
0,p0,course6,0
1,p0,course6,0
2,p0,course6,0
3,p0,course6,4
4,p1,course6,3
5,p1,course6,0
6,p1,course6,0
7,p1,course6,0
8,p2,course6,0
9,p2,course6,3


In [61]:
data.query("""
SELECT 
    table_name,
    partition_name,
    subpartition_name,
    table_rows
FROM 
    information_schema.partitions 
WHERE 
    table_name = 'course6';
""")

,TABLE_NAME,PARTITION_NAME,SUBPARTITION_NAME,TABLE_ROWS
0,course6,p0,p0sp0,0
1,course6,p0,p0sp1,0
2,course6,p0,p0sp2,0
3,course6,p0,p0sp3,4
4,course6,p1,p1sp0,3
5,course6,p1,p1sp1,0
6,course6,p1,p1sp2,0
7,course6,p1,p1sp3,0
8,course6,p2,p2sp0,0
9,course6,p2,p2sp1,3


In [67]:
data.query("""
SELECT * FROM course6 PARTITION (p0);
""")

,course_id,course_name,course_title,course_desc,launch_date,course_fee,course_mentor,course_launch_year
0,100,ML,Machine Learning,End to End Machine Learning course with captso...,2019-06-07,3999.990000,person1,2019
1,101,ML1.0,Machine Learning,End to End Machine Learning course with captso...,2019-07-07,3999.990000,person2,2019
2,102,ML2.0,Machine Learning,End to End Machine Learning course with captso...,2019-08-07,3999.990000,person3,2019
3,103,Ml3.0,Machine Learning,End to End Machine Learning course with captso...,2019-09-07,3999.990000,person1,2019
